In [1]:
# CÉLULA 1: INSTALAR AS BIBLIOTECAS (OpenCV e Scikit-learn)
!pip install opencv-python numpy scikit-learn

In [1]:
# CÉLULA 3: CÓDIGO FINAL (OpenCV DNN + SVM - Download Manual)
# (Execute APENAS esta célula após reiniciar)

import cv2
import os
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from google.colab import drive
import time

# 1. Montar o Google Drive
drive.mount('/content/drive')

# --- CONFIGURAÇÃO DOS CAMINHOS ---
base_dir = '/content/drive/MyDrive/Projeto_Deteccao_Video/' # Seu nome de pasta
dataset_path = os.path.join(base_dir, 'dataset_faces')
models_dir = os.path.join(base_dir, 'models_pretreinados')

# Vídeo de entrada e saída
video_input_path = os.path.join(base_dir, 'videos_entrada/interview_test.mp4')
video_output_path = os.path.join(base_dir, 'videos_saida/resultado_OpenCV_DNN_SVM.mp4')

# --- 2. DEFINIR CAMINHOS DOS MODELOS (QUE VOCÊ FEZ UPLOAD) ---
# Caminho ajustado para o link que você encontrou
prototxt_path = os.path.join(models_dir, 'deploy.prototxt')
caffemodel_path = os.path.join(models_dir, 'res10_300x300_ssd_iter_140000.caffemodel')
embedder_path = os.path.join(models_dir, 'openface_nn4.small2.v1.t7')


# --- 3. TREINAMENTO (APRENDER OS ROSTOS DE BELLA E PEDRO) ---
print("[INFO] Carregando modelos que você baixou manualmente...")
try:
    # Verificação de segurança
    if not os.path.exists(prototxt_path) or os.path.getsize(prototxt_path) < 1000:
        raise FileNotFoundError(f"Arquivo não encontrado ou corrompido: {prototxt_path}")
    if not os.path.exists(caffemodel_path) or os.path.getsize(caffemodel_path) < 1000000:
        raise FileNotFoundError(f"Arquivo não encontrado ou corrompido: {caffemodel_path}")
    if not os.path.exists(embedder_path) or os.path.getsize(embedder_path) < 5000000:
        raise FileNotFoundError(f"Arquivo não encontrado ou corrompido: {embedder_path}")

    detector = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)
    embedder = cv2.dnn.readNetFromTorch(embedder_path)
    print("[INFO] Modelos carregados com sucesso!")

except Exception as e:
    print(f"ERRO CRÍTICO AO CARREGAR OS MODELOS: {e}")
    print("Verifique se os 3 arquivos ('deploy.prototxt', 'res10_300x300_ssd_iter_140000.caffemodel', 'openface_nn4.small2.v1.t7')")
    print("estão com os nomes corretos e dentro da pasta 'models_pretreinados'.")
    raise SystemExit("Parando a execução.")


print("[INFO] Processando dataset_faces...")
known_embeddings = []
known_names = []

for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_folder):
        continue

    print(f"[INFO] Processando a pasta: {person_name}")
    for image_name in os.listdir(person_folder):
        image_path = os.path.join(person_folder, image_name)

        try:
            image = cv2.imread(image_path)
            if image is None:
                print(f"  [AVISO] Não foi possível ler a imagem {image_name}")
                continue

            (h, w) = image.shape[:2]

            # Detectar rosto na imagem de treino
            blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
            detector.setInput(blob)
            detections = detector.forward()

            if len(detections) > 0:
                i = np.argmax(detections[0, 0, :, 2])
                confidence = detections[0, 0, i, 2]

                if confidence > 0.5:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    face = image[startY:endY, startX:endX]

                    # Extrair o embedding (impressão digital)
                    faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
                    embedder.setInput(faceBlob)
                    vec = embedder.forward()

                    known_embeddings.append(vec.flatten())
                    known_names.append(person_name)
        except Exception as e:
            print(f"  [AVISO] Erro ao processar {image_name}: {e}")

print(f"[INFO] {len(known_names)} rostos de treino carregados.")

# Treinar o classificador SVM
print("[INFO] Treinando o classificador SVM...")
if len(known_names) < 2:
    print("ERRO CRÍTICO: Não foi possível carregar dados de treino suficientes (precisa de pelo menos 2 pessoas no dataset). Verifique sua pasta 'dataset_faces'.")
    raise SystemExit("Parando a execução.")

le = LabelEncoder()
labels = le.fit_transform(known_names)
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(known_embeddings, labels)
print("[INFO] Treinamento concluído.")

# --- 4. PROCESSAR O VÍDEO ---
print(f"[INFO] Abrindo o arquivo de vídeo: {video_input_path}")
cap = cv2.VideoCapture(video_input_path)

if not cap.isOpened():
    print("ERRO: Não foi possível abrir o arquivo de vídeo.")
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

    print(f"[INFO] Processando vídeo... Total de frames: {total_frames}. (Modo CPU, será lento).")
    start_time = time.time()
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        (h, w) = frame.shape[:2]

        # Detectar rostos no frame do vídeo
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        detector.setInput(blob)
        detections = detector.forward()

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            if confidence > 0.6: # Limiar de detecção
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                startX, startY = max(0, startX), max(0, startY)
                endX, endY = min(w - 1, endX), min(h - 1, endY)

                face = frame[startY:endY, startX:endX]

                if face.shape[0] > 0 and face.shape[1] > 0:
                    # Extrair o embedding
                    faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
                    embedder.setInput(faceBlob)
                    vec = embedder.forward()

                    # Fazer a predição (reconhecimento)
                    preds = recognizer.predict_proba(vec)[0]
                    j = np.argmax(preds)
                    proba = preds[j]

                    if proba > 0.65: # Limiar de reconhecimento
                        name = le.classes_[j]
                        text = f"{name}: {proba * 100:.2f}%"
                    else:
                        name = "Desconhecido"
                        text = name

                    # Desenhar a caixa e o nome
                    y = startY - 10 if startY - 10 > 10 else startY + 10
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    cv2.putText(frame, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        writer.write(frame)

        if frame_count % 50 == 0:
            print(f"[INFO] Frames processados: {frame_count} / {total_frames}")

    # --- FINALIZAÇÃO ---
    end_time = time.time()
    print("[INFO] Processamento concluído.")
    print(f"[INFO] Tempo total: {end_time - start_time:.2f} segundos.")
    cap.release()
    writer.release()

    print(f"[INFO] SUCESSO! O vídeo processado foi salvo em: {video_output_path}")

Mounted at /content/drive
[INFO] Carregando modelos que você baixou manualmente...
[INFO] Modelos carregados com sucesso!
[INFO] Processando dataset_faces...
[INFO] Processando a pasta: Bella_ramsey
[INFO] Processando a pasta: Pedro_Pascal
[INFO] 62 rostos de treino carregados.
[INFO] Treinando o classificador SVM...
[INFO] Treinamento concluído.
[INFO] Abrindo o arquivo de vídeo: /content/drive/MyDrive/Projeto_Deteccao_Video/videos_entrada/interview_test.mp4
[INFO] Processando vídeo... Total de frames: 2104. (Modo CPU, será lento).
[INFO] Frames processados: 50 / 2104
[INFO] Frames processados: 100 / 2104
[INFO] Frames processados: 150 / 2104
[INFO] Frames processados: 200 / 2104
[INFO] Frames processados: 250 / 2104
[INFO] Frames processados: 300 / 2104
[INFO] Frames processados: 350 / 2104
[INFO] Frames processados: 400 / 2104
[INFO] Frames processados: 450 / 2104
[INFO] Frames processados: 500 / 2104
[INFO] Frames processados: 550 / 2104
[INFO] Frames processados: 600 / 2104
[INFO]